In [1]:
import streamlit as st

In [64]:
%%writefile Loans_Classification_App.py

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import streamlit as st
import joblib


def getInput():
    
    person_age = st.slider('what is your age'.title() ,  min_value=20, max_value=70, step=1)
    person_gender =  st.selectbox('select gender'.title() , ['male', 'female'])
    person_education = st.selectbox('What type of your education'.title(),['Master' ,'High School' ,'Bachelor' ,'Associate' ,'Doctorate'])
    person_income =  st.slider('what is your income'.title() , min_value=8000.000 , max_value=2448661.0 , step=1000.0)
    person_emp_exp = st.slider('what is your years of Exp.'.title() , min_value=0 , max_value=50 , step=1)
    loan_amnt = st.slider('what is your Loan Amount'.title() , min_value=500.0 , max_value=35000.0 , step=100.0)
    person_home_ownership = st.selectbox('what is your home ownership'.title() ,['RENT' ,'OWN', 'MORTGAGE' ,'OTHER'])
    loan_intent = st.selectbox('what is your intention'.title() , ['PERSONAL' ,'EDUCATION' ,'MEDICAL' ,'VENTURE', 'HOMEIMPROVEMENT',
                                                                     'DEBTCONSOLIDATION'])
    loan_int_rate = st.slider('what is the rate of interest'.title(),min_value=5.42 , max_value=20.0 , step=0.1)
    loan_percent_income = st.slider('what is the percentile from income'.title(),min_value=0.0,max_value=0.62 , step=0.01)
    cb_person_cred_hist_length = st.slider('what is your Credit history years'.title(),min_value=2.0 ,max_value=30.0 , step=1.0)
    credit_score = st.slider('what is your Credit Score'.title(), min_value=418 ,max_value=772 , step=5)
    previous_loan_defaults_on_file = st.selectbox('Previous Loan Accepted or Denied'.title() , [0 ,1])


    return pd.DataFrame(
        data=[ 
            [person_age, person_gender, person_education, person_income, person_emp_exp,loan_amnt,
       person_home_ownership, loan_intent,loan_int_rate,
       loan_percent_income, cb_person_cred_hist_length, credit_score,previous_loan_defaults_on_file]
        ] , 
                 columns=['person_age', 'person_gender', 'person_education', 'person_income', 'person_emp_exp',
       'loan_amnt', 'person_home_ownership', 'loan_intent',
       'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length' , 'credit_score' , 'previous_loan_defaults_on_file'])

test = getInput()
st.dataframe(test)
model = joblib.load('model.h5')

st.write('Accepted' if model.predict(test) == 1 else 'Not Accepted')


Overwriting Loans_Classification_App.py


In [65]:
! streamlit run Loans_Classification_App.py

^C


In [17]:
import pandas as pd

In [4]:
df=pd.read_csv("loan_data.csv")
df

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,22.0,female,Master,71948.0,0,RENT,35000.0,PERSONAL,16.02,0.49,3.0,561,No,1
1,21.0,female,High School,12282.0,0,OWN,1000.0,EDUCATION,11.14,0.08,2.0,504,Yes,0
2,25.0,female,High School,12438.0,3,MORTGAGE,5500.0,MEDICAL,12.87,0.44,3.0,635,No,1
3,23.0,female,Bachelor,79753.0,0,RENT,35000.0,MEDICAL,15.23,0.44,2.0,675,No,1
4,24.0,male,Master,66135.0,1,RENT,35000.0,MEDICAL,14.27,0.53,4.0,586,No,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44995,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
44996,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
44997,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
44998,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [5]:
for col in df.select_dtypes('object').columns[1:] :
    
    print(col)
    print(df[col].unique() )

person_education
['Master' 'High School' 'Bachelor' 'Associate' 'Doctorate']
person_home_ownership
['RENT' 'OWN' 'MORTGAGE' 'OTHER']
loan_intent
['PERSONAL' 'EDUCATION' 'MEDICAL' 'VENTURE' 'HOMEIMPROVEMENT'
 'DEBTCONSOLIDATION']
previous_loan_defaults_on_file
['No' 'Yes']


In [9]:
from imblearn.under_sampling import RandomUnderSampler

# Features (X) and target (y)
X = df.drop('loan_status', axis=1)  # Drop the target column
y = df['loan_status']              # Target column

# Initialize undersampler
rus = RandomUnderSampler(random_state=42)

# Apply undersampling
X_resampled, y_resampled = rus.fit_resample(X, y)

# Combine back into a DataFrame (optional)
df_balanced = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled, columns=['loan_status'])], axis=1)

In [11]:
# Drop rows where 'person_age' is greater than 70 and reset the index
df_balanced = df_balanced[df_balanced['person_age'] <= 70].reset_index(drop=True)
df_balanced

,person_age,person_gender,person_education,person_income,person_emp_exp,person_home_ownership,loan_amnt,loan_intent,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,previous_loan_defaults_on_file,loan_status
0,28.0,male,Associate,72926.0,2,MORTGAGE,16800.0,DEBTCONSOLIDATION,12.61,0.23,7.0,586,Yes,0
1,23.0,female,Associate,72988.0,0,MORTGAGE,4500.0,MEDICAL,7.40,0.06,3.0,562,Yes,0
2,22.0,male,Associate,78823.0,0,MORTGAGE,4000.0,DEBTCONSOLIDATION,7.14,0.05,4.0,624,Yes,0
3,23.0,male,Bachelor,79971.0,0,RENT,12000.0,VENTURE,11.01,0.15,4.0,671,No,0
4,28.0,female,High School,80906.0,5,RENT,16000.0,VENTURE,12.18,0.20,6.0,642,Yes,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19990,27.0,male,Associate,47971.0,6,RENT,15000.0,MEDICAL,15.66,0.31,3.0,645,No,1
19991,37.0,female,Associate,65800.0,17,RENT,9000.0,HOMEIMPROVEMENT,14.07,0.14,11.0,621,No,1
19992,33.0,male,Associate,56942.0,7,RENT,2771.0,DEBTCONSOLIDATION,10.02,0.05,10.0,668,No,1
19993,29.0,male,Bachelor,33164.0,4,RENT,12000.0,EDUCATION,13.23,0.36,6.0,604,No,1


In [12]:
df_balanced.select_dtypes('number').describe()

,person_age,person_income,person_emp_exp,loan_amnt,loan_int_rate,loan_percent_income,cb_person_cred_hist_length,credit_score,loan_status
count,19995.000000,1.999500e+04,19995.000000,19995.000000,19995.000000,19995.000000,19995.000000,19995.000000,19995.000000
mean,27.668317,7.229944e+04,5.320680,10014.815404,11.673256,0.162152,5.835559,632.087822,0.500125
std,5.928192,5.636485e+04,5.944779,6626.237560,3.130796,0.098932,3.889615,50.495494,0.500012
min,20.000000,8.000000e+03,0.000000,500.000000,5.420000,0.000000,2.000000,418.000000,0.000000
25%,24.000000,4.174950e+04,1.000000,5000.000000,9.630000,0.080000,3.000000,601.000000,0.000000
50%,26.000000,6.101300e+04,4.000000,8500.000000,11.360000,0.140000,4.000000,639.000000,1.000000
75%,30.000000,8.741500e+04,8.000000,14000.000000,13.980000,0.230000,8.000000,669.000000,1.000000
max,70.000000,2.448661e+06,50.000000,35000.000000,20.000000,0.620000,30.000000,772.000000,1.000000
